In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по КРС.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,КРС
614,ГАЛМАТЫ,2016-03-01,94.38
584,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2023-09-01,4743.50
1830,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2023-03-01,3157.46
1002,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2021-12-01,18239.40
239,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-12-01,10603.99
1474,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2021-04-01,1663.80
735,ГАСТАНА,2016-04-01,9.89
37,АКМОЛИНСКАЯ ОБЛАСТЬ,2018-02-01,4100.47
365,АТЫРАУСКАЯ ОБЛАСТЬ,2015-06-01,1946.39
116,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-09-01,2909.66


In [8]:
regions = df['Регион'].unique()
target   = "КРС"
horizon  = 3
epsilon = 1e-6

## Holt-Winters

In [9]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

In [5]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        model = ExponentialSmoothing(
            train,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты пронозов на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winters по регионам (rolling-3):")
display(final_hw)

Результаты пронозов на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[4133.83, 3442.91, 4009.72]","[4035.37, 3446.55, 3779.21]",144.733915,110.870577,2.881671
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[3408.39, 3974.56, 3617.58]","[3446.55, 3779.21, 3691.05]",122.493482,102.325695,2.755561
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[3952.57, 3606.75, 3632.84]","[3779.21, 3691.05, 3574.86]",116.218621,105.211995,2.830961
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[3562.33, 3561.29, 3897.55]","[3691.05, 3574.86, 3951.41]",80.939247,65.383912,1.743354
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[3623.22, 3971.56, 1686.98]","[3574.86, 3951.41, 2029.47]",200.039163,136.999626,6.246213
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[7927.08, 7395.72, 9299.72]","[15715.54, 6948.86, 9372.79]",4504.264280,2769.464184,18.923096
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[8947.35, 10838.01, 11632.38]","[6948.86, 9372.79, 15738.07]",2768.723557,2523.132693,23.493433
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[9876.09, 10689.04, 11750.0]","[9372.79, 15738.07, 10566.5]",3008.138579,2245.277471,16.217317
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[10852.45, 11588.44, 11624.79]","[15738.07, 10566.5, 11441.98]",2883.693646,2030.122042,14.104168


Средние метрики Holt–Winters по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,235.27,200.03,7.62
1,АКТЮБИНСКАЯ ОБЛАСТЬ,964.89,842.97,15.52
2,АЛМАТИНСКАЯ ОБЛАСТЬ,1138.56,976.08,21.12
3,АТЫРАУСКАЯ ОБЛАСТЬ,229.38,204.48,13.45
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1891.76,1844.47,63.70
5,ГАЛМАТЫ,13.44,11.55,694.06
6,ГАСТАНА,1.08,0.92,35.79
7,ГШЫМКЕНТ,416.06,371.04,83.19
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,507.70,480.06,13.59
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,294.43,246.51,5.86


In [10]:
regions

array(['АКМОЛИНСКАЯ ОБЛАСТЬ', 'АКТЮБИНСКАЯ ОБЛАСТЬ',
       'АЛМАТИНСКАЯ ОБЛАСТЬ', 'АТЫРАУСКАЯ ОБЛАСТЬ',
       'ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ГАЛМАТЫ', 'ГАСТАНА', 'ГШЫМКЕНТ',
       'ЖАМБЫЛСКАЯ ОБЛАСТЬ', 'ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ',
       'КАРАГАНДИНСКАЯ ОБЛАСТЬ', 'КОСТАНАЙСКАЯ ОБЛАСТЬ',
       'КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ', 'МАНГИСТАУСКАЯ ОБЛАСТЬ', 'ОБЛАСТЬ АБАЙ',
       'ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ', 'ПАВЛОДАРСКАЯ ОБЛАСТЬ',
       'СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ТУРКЕСТАНСКАЯ ОБЛАСТЬ'],
      dtype=object)

## Holw-Winter's (log)

In [11]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[4059.46, 3449.58, 3899.33]","[4035.37, 3446.55, 3779.21]",70.755483,49.081411,1.287815
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[3441.1, 3889.78, 3600.63]","[3446.55, 3779.21, 3691.05]",82.523185,68.810703,1.844455
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[3892.82, 3603.56, 3628.19]","[3779.21, 3691.05, 3574.86]",88.326413,84.808585,2.289403
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[3558.51, 3583.37, 3877.67]","[3691.05, 3574.86, 3951.41]",87.705400,71.597100,1.898370
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[3638.69, 3937.14, 1900.76]","[3574.86, 3951.41, 2029.47]",83.352679,68.934890,2.829502
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[7842.97, 7289.94, 8800.3]","[15715.54, 6948.86, 9372.79]",4561.484633,2928.713189,20.370201
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[7926.49, 9556.52, 10605.77]","[6948.86, 9372.79, 15738.07]",3018.278853,2097.886801,16.213301
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[9381.81, 10415.31, 11429.82]","[9372.79, 15738.07, 10566.5]",3113.262835,2065.033537,14.029166
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[10415.3, 11429.26, 11406.99]","[15738.07, 10566.5, 11441.98]",3113.275566,2073.507419,14.097286


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,153.57,129.92,4.77
1,АКТЮБИНСКАЯ ОБЛАСТЬ,778.20,600.08,10.54
2,АЛМАТИНСКАЯ ОБЛАСТЬ,1064.41,880.87,12.93
3,АТЫРАУСКАЯ ОБЛАСТЬ,169.73,148.49,8.87
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,414.51,357.77,9.46
5,ГАЛМАТЫ,3.84,2.82,100.69
6,ГАСТАНА,0.46,0.40,15.60
7,ГШЫМКЕНТ,232.30,203.83,43.10
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,497.74,430.01,10.74
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,260.95,218.99,5.22


## SARIMA

In [ ]:
# regions1to9 = regions[:9]
# regions10to14 = regions[9:14]
# regions15to20 = regions[14:]


In [12]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(0, 1, 2)","(0, 1, 1, 12)",98.14,78.86,2.09,"[3893.47, 3352.97, 3778.1]","[4035.37, 3446.55, 3779.21]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(0, 1, 2)","(0, 1, 1, 12)",86.67,72.96,1.96,"[3454.4, 3894.89, 3595.7]","[3446.55, 3779.21, 3691.05]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(0, 1, 2)","(0, 1, 1, 12)",85.89,75.91,2.04,"[3890.09, 3593.82, 3594.47]","[3779.21, 3691.05, 3574.86]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(0, 1, 2)","(2, 1, 0, 12)",188.55,186.77,4.98,"[3496.1, 3423.01, 3737.89]","[3691.05, 3574.86, 3951.41]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(0, 1, 2)","(0, 1, 1, 12)",87.86,79.10,3.07,"[3656.87, 3920.52, 1905.06]","[3574.86, 3951.41, 2029.47]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 1, 2)","(0, 1, 1, 12)",5706.81,3686.53,25.91,"[5868.41, 6375.49, 10011.88]","[15715.54, 6948.86, 9372.79]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(1, 0, 0)","(0, 1, 1, 12)",2955.49,2748.27,25.56,"[8881.75, 11399.94, 11453.29]","[6948.86, 9372.79, 15738.07]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(0, 0, 0)","(0, 1, 1, 12)",2735.27,1989.05,14.60,"[10634.39, 11173.69, 10707.65]","[9372.79, 15738.07, 10566.5]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(0, 0, 0)","(0, 1, 1, 12)",2702.99,1798.24,12.09,"[11086.6, 10883.37, 11015.59]","[15738.07, 10566.5, 11441.98]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,170.89,146.60,5.22
1,АКТЮБИНСКАЯ ОБЛАСТЬ,887.97,728.64,12.77
2,АЛМАТИНСКАЯ ОБЛАСТЬ,918.74,730.59,11.63
3,АТЫРАУСКАЯ ОБЛАСТЬ,160.94,142.53,8.61
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,439.20,383.63,10.50
5,ГАЛМАТЫ,4.46,3.69,87.36
6,ГАСТАНА,0.79,0.71,26.88
7,ГШЫМКЕНТ,204.56,135.17,28.85
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,415.41,360.33,9.24
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,238.56,206.83,4.57


In [ ]:
results_sarima = []

for region in regions1to9:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12
        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         list(actual)
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
results_sarima = []

for region in regions10to14:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12
        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         list(actual)
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
results_sarima = []

for region in regions15to20:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         list(actual)
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
regions12 = regions[:2]
regions12

In [ ]:
results_sarima = []

for region in regions12:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    # пропускаем, если слишком мало данных
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        # проверяем полноту окон
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # автоподбор SARIMA
        sarima = auto_arima(
            train,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        fc = sarima.predict(n_periods=horizon, return_conf_int=False)


        # метрики

        if np.isnan(fc).any() or test.isna().any():
            print(f"{region} @ {test_start.strftime('%Y-%m')}: есть NaN, пропускаем окно.")
            continue
        
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test.values - fc) / test.values).mean()) * 100

        results_sarima.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "order":       sarima.order,
            "seasonal_order": sarima.seasonal_order,
            "RMSE":        round(rmse,2),
            "MAE":         round(mae,2),
            "MAPE_%":      round(mape,2),
            "Forecast":    [round(x,2) for x in fc],
            "Actual":      list(test.values)
        })


# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
regions34 = regions[2:4]
regions34

In [ ]:
results_sarima = []

for region in regions34:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    # пропускаем, если слишком мало данных
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        # проверяем полноту окон
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # автоподбор SARIMA
        sarima = auto_arima(
            train,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        fc = sarima.predict(n_periods=horizon, return_conf_int=False)


        # метрики

        if np.isnan(fc).any() or test.isna().any():
            print(f"{region} @ {test_start.strftime('%Y-%m')}: есть NaN, пропускаем окно.")
            continue
        
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test.values - fc) / test.values).mean()) * 100

        results_sarima.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "order":       sarima.order,
            "seasonal_order": sarima.seasonal_order,
            "RMSE":        round(rmse,2),
            "MAE":         round(mae,2),
            "MAPE_%":      round(mape,2),
            "Forecast":    [round(x,2) for x in fc],
            "Actual":      list(test.values)
        })


# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
regions56 = regions[4:6]
regions56

In [ ]:
results_sarima = []

for region in regions56:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    # пропускаем, если слишком мало данных
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        # проверяем полноту окон
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # автоподбор SARIMA
        sarima = auto_arima(
            train,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        fc = sarima.predict(n_periods=horizon, return_conf_int=False)


        # метрики

        if np.isnan(fc).any() or test.isna().any():
            print(f"{region} @ {test_start.strftime('%Y-%m')}: есть NaN, пропускаем окно.")
            continue
        
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test.values - fc) / test.values).mean()) * 100

        results_sarima.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "order":       sarima.order,
            "seasonal_order": sarima.seasonal_order,
            "RMSE":        round(rmse,2),
            "MAE":         round(mae,2),
            "MAPE_%":      round(mape,2),
            "Forecast":    [round(x,2) for x in fc],
            "Actual":      list(test.values)
        })


# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
results_sarima = []

for region in regions56:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        sarima_log = auto_arima(
            train_log,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         list(actual)
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
regions78 = regions[6:8]
regions78

In [ ]:
results_sarima = []

for region in regions78:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    # пропускаем, если слишком мало данных
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        # проверяем полноту окон
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # автоподбор SARIMA
        sarima = auto_arima(
            train,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        fc = sarima.predict(n_periods=horizon, return_conf_int=False)


        # метрики

        if np.isnan(fc).any() or test.isna().any():
            print(f"{region} @ {test_start.strftime('%Y-%m')}: есть NaN, пропускаем окно.")
            continue
        
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test.values - fc) / test.values).mean()) * 100

        results_sarima.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "order":       sarima.order,
            "seasonal_order": sarima.seasonal_order,
            "RMSE":        round(rmse,2),
            "MAE":         round(mae,2),
            "MAPE_%":      round(mape,2),
            "Forecast":    [round(x,2) for x in fc],
            "Actual":      list(test.values)
        })


# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
results_sarima = []

for region in regions78:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        sarima_log = auto_arima(
            train_log,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         list(actual)
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
regions910 = regions[8:10]
regions910

In [ ]:
results_sarima = []

epsilon = 1e-6  # чтобы не было log(0)

for region in regions910:
    # исходный ts
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )

    # добавляем epsilon и логарифмируем
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        sarima_log = auto_arima(
            train_log,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         list(actual)
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

In [ ]:
results

In [ ]:
assert not train.isna().any()


In [ ]:
hw_results=[]
for region in regions:
    ts = (df[df["Регион"]==region].set_index("Период")[target].dropna().sort_index())
    n=len(ts)
    if n<24:
        continue
    for i in range(horizon, n):
        train=ts.iloc[:i]
        test =ts.iloc[i:i+horizon]
        if len(test)<horizon: break
        hw=ExponentialSmoothing(train, seasonal="add", seasonal_periods=12).fit()
        fc=hw.forecast(horizon)
        hw_results.append({
            "Регион": region,
            "Окно начала": train.index[-1],
            "RMSE": np.sqrt(mean_squared_error(test,fc)),
            "MAE":  mean_absolute_error(test,fc),
            "MAPE": (np.abs((test-fc)/test).mean())*100
        })

hw_df=pd.DataFrame(hw_results).groupby("Регион").mean().reset_index()
# display(hw_df)

In [ ]:
# контейнер для итоговых усреднённых метрик
agg = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    n = len(ts)
    if n < min(min_train_hw, min_train_other):
        print(f"{region}: слишком мало данных ({n} мес.), пропуск.")
        continue

    # накопители метрик
    acc = {
        "Holt-Winters": {"RMSE": [], "MAE": [], "MAPE": []},
        "Auto-ARIMA":   {"RMSE": [], "MAE": [], "MAPE": []},
        "Prophet":      {"RMSE": [], "MAE": [], "MAPE": []},
    }

    # скользящий forecast
    for i in range(horizon, n):
        train = ts.iloc[:i]
        test  = ts.iloc[i : i+horizon]
        if len(test) < horizon:
            break

        # --- Holt-Winters ---
        if len(train) >= min_train_hw:
            hw = ExponentialSmoothing(
                train, seasonal="add", seasonal_periods=12
            ).fit(optimized=True)
            hw_fc = hw.forecast(horizon)
            acc["Holt-Winters"]["RMSE"].append(
                np.sqrt(mean_squared_error(test, hw_fc))
            )
            acc["Holt-Winters"]["MAE"].append(
                mean_absolute_error(test, hw_fc)
            )
            acc["Holt-Winters"]["MAPE"].append(
                (np.abs((test - hw_fc) / test).mean()) * 100
            )


In [ ]:
# Задайте название целевого столбца:
target = "КРС"

# Горизонт прогноза (количество отстающих месяцев)
horizon = 3

regions = df["Регион"].unique()

# --- 3. Holt-Winters ---
print("## Holt-Winters\n")
hw_summary = []
for region in regions:
    ts = df[df["Регион"] == region].set_index("Период")[target].dropna()
    if len(ts) < 24:
        print(f"- {region}: недостаточно данных (<24 мес.), пропускаем.")
        continue
    train, test = ts.iloc[:-horizon], ts.iloc[-horizon:]
    hw = ExponentialSmoothing(train, seasonal='add', seasonal_periods=12).fit()
    forecast = hw.forecast(horizon)
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mae = mean_absolute_error(test, forecast)
    mape = (np.abs((test - forecast) / test).mean()) * 100
    hw_summary.append({
        "Регион": region,
        "Метод": "Holt-Winters",
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "Прогноз": [round(x, 2) for x in forecast.values],
        "Факт": list(test.values),
        "CI_lower": [np.nan]*horizon,
        "CI_upper": [np.nan]*horizon
    })
hw_df = pd.DataFrame(hw_summary)
display(hw_df)


In [ ]:
horizon = 3
results = []

# допустим, периоды у нас от 2015-01 до 2024-12
start = pd.to_datetime("2015-01-01")
end   = pd.to_datetime("2024-12-01")

curr = start
while curr + pd.DateOffset(months=horizon) - 1 <= end:
    test_start = curr
    test_end   = curr + pd.DateOffset(months=horizon) - 1

    train = ts[ts.index < test_start]
    test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

    # обучаем и предсказываем, считаем метрики...
    results.append({
      "window_start": test_start,
      "RMSE": rmse, 
      "MAE": mae, 
      "MAPE": mape
    })

    # сдвигаемся на 1 месяц
    curr += pd.DateOffset(months=1)

# усредняем по всем «оконцам»
df_res = pd.DataFrame(results)
print("Средний MAPE:", df_res["MAPE"].mean())


In [ ]:

# --- 4. SARIMA ---
print("\n## SARIMA\n")
sarima_summary = []
for region in regions:
    ts = df[df["Регион"] == region].set_index("Период")[target].dropna()
    if len(ts) < horizon + 12:
        print(f"- {region}: недостаточно данных, пропускаем.")
        continue
    train, test = ts.iloc[:-horizon], ts.iloc[-horizon:]
    model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12)).fit(disp=False)
    fc_res = model.get_forecast(steps=horizon)
    forecast = fc_res.predicted_mean
    ci = fc_res.conf_int(alpha=0.05)
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mae = mean_absolute_error(test, forecast)
    mape = (np.abs((test - forecast) / test).mean()) * 100
    sarima_summary.append({
        "Регион": region,
        "Метод": "SARIMA",
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "Прогноз": list(forecast.values),
        "Факт": list(test.values),
        "CI_lower": ci.iloc[:,0].tolist(),
        "CI_upper": ci.iloc[:,1].tolist()
    })
sarima_df = pd.DataFrame(sarima_summary)
display(sarima_df)

# --- 5. Prophet ---
print("\n## Prophet\n")
prophet_summary = []
for region in regions:
    tmp = df[df["Регион"] == region][["Период", target]].dropna()
    if len(tmp) < horizon + 12:
        print(f"- {region}: недостаточно данных, пропускаем.")
        continue
    train = tmp.iloc[:-horizon].rename(columns={"Период":"ds", target:"y"})
    test = tmp.iloc[-horizon:].copy()
    m = Prophet(yearly_seasonality=True)
    m.fit(train)
    future = m.make_future_dataframe(periods=horizon, freq="M")
    fc = m.predict(future).set_index("ds")
    forecast = fc["yhat"].iloc[-horizon:]
    rmse = np.sqrt(mean_squared_error(test[target].values, forecast))
    mae = mean_absolute_error(test[target].values, forecast)
    mape = (np.abs((test[target].values - forecast)/test[target].values).mean()) * 100
    prophet_summary.append({
        "Регион": region,
        "Метод": "Prophet",
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "Прогноз": forecast.tolist(),
        "Факт": test[target].tolist(),
        "CI_lower": fc["yhat_lower"].iloc[-horizon:].tolist(),
        "CI_upper": fc["yhat_upper"].iloc[-horizon:].tolist()
    })
prophet_df = pd.DataFrame(prophet_summary)
display(prophet_df)

# --- 6. Итоговое сравнение ---
print("\n## Лучшие модели по регионам\n")
all_res = pd.concat([hw_df, sarima_df, prophet_df], ignore_index=True)
best = all_res.loc[all_res.groupby("Регион")["MAPE"].idxmin()].reset_index(drop=True)
display(best[["Регион","Метод","MAPE","Прогноз","CI_lower","CI_upper"]])